In [8]:
import requests
import time
import pandas as pd
from config import email, key
import matplotlib.pyplot as plt


In [9]:
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
times_url = "&starttime=1990-01-01"
magnitude_url ="&minmagnitude=5.5"
geometry_url = "&minlatitude=25&maxlatitude=50&minlongitude=-125&maxlongitude=-65"

search = requests.get(base_url + times_url + magnitude_url + geometry_url)
response = search.json()
response

{'type': 'FeatureCollection',
 'metadata': {'generated': 1570962249000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1990-01-01&minmagnitude=5.5&minlatitude=25&maxlatitude=50&minlongitude=-125&maxlongitude=-65',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.8.1',
  'count': 98},
 'features': [{'type': 'Feature',
   'properties': {'mag': 5.5,
    'place': '15km ESE of Little Lake, CA',
    'time': 1562384873420,
    'updated': 1569508224040,
    'tz': -480,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38457687',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38457687&format=geojson',
    'felt': 32,
    'cdi': 7.1,
    'mmi': 4.916,
    'alert': None,
    'status': 'reviewed',
    'tsunami': 0,
    'sig': 488,
    'net': 'ci',
    'code': '38457687',
    'ids': ',ci38457687,us70004bpa,',
    'sources': ',ci,us,',
    'types': ',dyfi,focal-mechanism,geoserve,moment-tensor,nearby-cities,orig

In [16]:
record_list = []
i=0

for record in response['features']:
    identifier = response['features'][i]['id']
    mag = response['features'][i]['properties']['mag']
    url = response['features'][i]['properties']['url']
    epochtime = response['features'][i]['properties']['time']
    converted_date= time.strftime('%Y%m%d',  time.gmtime(epochtime/1000))
    converted_time = time.strftime('%H:%M:%S',  time.gmtime(epochtime/1000))
    timezone=response['features'][i]['properties']['tz']
    place = response['features'][i]['properties']['place']
    significance=response['features'][i]['properties']['sig']
    lon = response['features'][i]['geometry']['coordinates'][0]
    lat = response['features'][i]['geometry']['coordinates'][1]
    depth = response['features'][i]['geometry']['coordinates'][2]
    record_dict = {"Identifier":identifier, "Location":place, "Magnitude":mag, "Date":converted_date,
                   "Time":converted_time,"Time Zone":timezone, "Significance":significance, 
                   "Lat":lat, "Lon":lon, "Depth":depth, "URL":url}
    record_list.append(record_dict)
    
    i += 1
    
record_df = pd.DataFrame(record_list)

record_df = record_df[~record_df["Location"].str.contains("Mexico")]

record_df = record_df[~record_df["Location"].str.contains("MX")].reset_index()

record_df = record_df.fillna(value="")

record_df

,index,Identifier,Location,Magnitude,Date,Time,Time Zone,Significance,Lat,Lon,Depth,URL
0,0,ci38457687,"15km ESE of Little Lake, CA",5.50,20190706,03:47:53,-480,488,35.901167,-117.749500,5.040,https://earthquake.usgs.gov/earthquakes/eventp...
1,1,ci38457511,2019 Ridgecrest Earthquake Sequence,7.10,20190706,03:19:53,-480,1596,35.769500,-117.599333,8.000,https://earthquake.usgs.gov/earthquakes/eventp...
2,2,ci38443183,"11km SW of Searles Valley, CA",6.40,20190704,17:33:49,-480,1460,35.705333,-117.503833,10.500,https://earthquake.usgs.gov/earthquakes/eventp...
3,3,nc73201181,"6km SSW of Petrolia, CA",5.58,20190623,03:53:02,-480,1139,40.273500,-124.300333,9.440,https://earthquake.usgs.gov/earthquakes/eventp...
4,5,us10009757,"11km SE of Lincoln, Montana",5.80,20170706,06:30:17,-420,1278,46.881100,-112.575300,12.180,https://earthquake.usgs.gov/earthquakes/eventp...
...,...,...,...,...,...,...,...,...,...,...,...,...
74,93,usp00050yv,Gulf of California,5.60,19911226,00:22:18,,482,25.307000,-109.693000,10.000,https://earthquake.usgs.gov/earthquakes/eventp...
75,94,nc228027,Northern California,6.00,19910817,19:29:40,,554,40.251667,-124.285833,8.320,https://earthquake.usgs.gov/earthquakes/eventp...
76,95,ci2021449,"13km NNE of Sierra Madre, CA",5.80,19910628,14:43:54,,636,34.270000,-117.993000,8.049,https://earthquake.usgs.gov/earthquakes/eventp...
77,96,nc20092556,Central California,5.80,19901024,06:15:19,,518,38.061833,-119.119167,10.903,https://earthquake.usgs.gov/earthquakes/eventp...


In [ ]:
record_df.to_csv("earthquakes.csv")